In [ ]:
# IMPORT LIBRARIES 
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, classification_report

In [ ]:
#LOAD THE DATASET 
df = pd.read_csv(r"C:\Users\sneha\Downloads\tool\heart.csv")
df.columns = df.columns.str.strip().str.replace(" ", "_")
X = df.drop(['HeartDisease'], axis=1)
y = df['HeartDisease']

In [ ]:
df.shape

In [ ]:
print(df.columns.tolist())


In [ ]:
#PREPROCESSING THE DATA 
from sklearn.preprocessing import LabelEncoder
encoders = {}
for col in X.columns:
    if X[col].dtype == 'object':
        le = LabelEncoder()
        X[col] = le.fit_transform(X[col])
        encoders[col] = le  # Save encoder for this column
print("Encoders created for:", list(encoders.keys()))

In [ ]:
#SPLIT THE DATA 
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [ ]:
#TRAIN THE MODEL
model = RandomForestClassifier()
model.fit(X_train, y_train)


In [ ]:
#PREDICT THE MODEL 
y_pred = model.predict(X_test)


In [ ]:
import joblib

# Save model
joblib.dump(model, "heart_model.pkl")

# Save encoders
joblib.dump(encoders, "encoders.pkl")


In [ ]:
#CHECK ACCURACY
print("Accuracy:", accuracy_score(y_test, y_pred))
print("\nClassification Report:\n", classification_report(y_test, y_pred))


In [ ]:
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
cm = confusion_matrix(y_test, y_pred)
ConfusionMatrixDisplay(confusion_matrix=cm).plot()


In [ ]:
import matplotlib.pyplot as plt

importances = model.feature_importances_
features = X.columns
plt.barh(features, importances)
plt.title("Feature Importance")
plt.show()


In [ ]:
print("Available Encoders:", list(encoders.keys()))


In [ ]:
import tkinter as tk
from tkinter import messagebox

# Prediction function
def predict():
    try:
        input_data = {
            'Age': int(entries["Age"].get()),
            'Sex': entries["Sex"].get(),
            'ChestPainType': entries["ChestPainType"].get(),
            'RestingBP': int(entries["RestingBP"].get()),
            'Cholesterol': int(entries["Cholesterol"].get()),
            'FastingBS': int(entries["FastingBS"].get()),
            'RestingECG': entries["RestingECG"].get(),
            'MaxHR': int(entries["MaxHR"].get()),
            'ExerciseAngina': entries["ExerciseAngina"].get(),
            'Oldpeak': float(entries["Oldpeak"].get()),
            'ST_Slope': entries["ST_Slope"].get()
        }

        new_df = pd.DataFrame([input_data])
        for col in new_df.columns:
            if col in encoders:
                new_df[col] = encoders[col].transform(new_df[col])

        prediction = model.predict(new_df)[0]
        result = "🚨 Heart Disease Detected" if prediction == 1 else "✅ No Heart Disease"
        messagebox.showinfo("Prediction", result)

        # Save to CSV
        new_df["Prediction"] = prediction
        new_df.to_csv("prediction_result.csv", mode='a', header=not pd.read_csv("prediction_result.csv").empty if "prediction_result.csv" in locals() else True, index=False)

    except Exception as e:
        messagebox.showerror("Error", str(e))

# Build GUI
root = tk.Tk()
root.title("Heart Disease Predictor")
root.geometry("400x650")

labels = ["Age", "Sex", "ChestPainType", "RestingBP", "Cholesterol", "FastingBS",
          "RestingECG", "MaxHR", "ExerciseAngina", "Oldpeak", "ST_Slope"]
entries = {}

for label in labels:
    tk.Label(root, text=label).pack()
    entry = tk.Entry(root)
    entry.pack()
    entries[label] = entry

tk.Button(root, text="Predict", command=predict).pack(pady=20)
root.mainloop()
